# Solving elasticity problems using JuliaFEM

Author(s): Jukka Aho

**Abstract**: A workflow to solve typical elasticity problem. This document also tries to give some quidelines how to develop JuliaFEM.

## Bottom-up design

We go piece by piece starting from something simple and going up to more complicated programming model.

*Design principle 1*: we introduce new ideas using Notebooks.

*Design principle 2*: we use ``Logging``. Forget ``println``.

In [1]:
using Logging
Logging.configure(level=DEBUG)

Logger(root,DEBUG,Pipe(open, 0 bytes waiting),root)

*Design principle 3*: we write docstrings using [numpy style](https://github.com/numpy/numpy/blob/master/doc/HOWTO_DOCUMENT.rst.txt).

*Design principle 4*: we don't use greek characters in code which is implemented to JuliaFEM. In notebooks they are ok.

*Design principle 5*: we use 4 space indentation like in Python.

First we write some elementary functions to calculate stiffness matrix.

In [2]:
"""
Calculate local tangent stiffness matrix and residual force vector
R = T - F for elasticity problem.

Parameters
----------
X : Element coordinates
u : Displacement field
R : Residual force vector
K : Tangent stiffness matrix
basis : Basis functions
dbasis : Derivative of basis functions
lambda : Material parameter
mu : Material parameter
ipoints : integration points
iweights : integration weights

Returns
-------
None

Notes
-----
If material parameters are given in list, they are interpolated to gauss
points using shape functions.

Examples
--------

"""
function calc_local_matrices!(X, u, R, K, basis, dbasis, lambda_, mu_, ipoints, iweights)
    dim, nnodes = size(X)
    I = eye(dim)
    R[:,:] = 0.0
    K[:,:] = 0.0

    dF = zeros(dim, dim)

    for m = 1:length(iweights)
        w = iweights[m]
        xi = ipoints[m, :]
        # calculate material parameters
        lambda = typeof(lambda_) == Float64 ? lambda_ : dot(lambda_, basis(xi))
        mu = typeof(mu_) == Float64 ? mu_ : dot(mu_, basis(xi))
        Jt = X*dbasis(xi)
        detJ = det(Jt)
        dbasisdX = dbasis(xi)*inv(Jt)

        gradu = u*dbasisdX
        F = I + gradu  # Deformation gradient
        E = 1/2*(gradu' + gradu + gradu'*gradu)  # Green-Lagrange strain tensor
        S = lambda*trace(E)*I + 2*mu*E  # PK2 stress tensor
        P = F*S  # PK1 stress tensor

        R[:,:] += w*P*dbasisdX'*detJ

        for p = 1:nnodes
            for i = 1:dim
                dF[:,:] = 0.0
                dF[i,:] = dbasisdX[p,:]
                dE = 1/2*(F'*dF + dF'*F)
                dS = lambda*trace(dE)*I + 2*mu*dE
                dP = dF*S + F*dS
                for q = 1:nnodes
                    for j = 1:dim
                        K[dim*(p-1)+i,dim*(q-1)+j] += w*(dP[j,:]*dbasisdX[q,:]')[1]*detJ
                    end
                end
            end
        end

    end
end

calc_local_matrices! (generic function with 1 method)

*Design principle 6*: we test our code. We use FactCheck for testing.

In [3]:
using FactCheck

In [4]:
facts("test solve one element model") do
    X = [0.0 0.0; 10.0 0.0; 10.0 1.0; 0.0 1.0]'
    F = [0 0; 0 0; 0 -2; 0 0]'

    # Material properties
    E = 90
    nu = 0.25
    mu = E/(2*(1+nu))
    la = E*nu/((1+nu)*(1-2*nu))
    la = 2*la*mu/(la + 2*mu)

    u = zeros(2, 4)
    du = zeros(2, 4)
    R = zeros(2, 4)
    K = zeros(8, 8)

    basis(xi) = [
        (1-xi[1])*(1-xi[2])/4
        (1+xi[1])*(1-xi[2])/4
        (1+xi[1])*(1+xi[2])/4
        (1-xi[1])*(1+xi[2])/4]

    dbasis(xi) = [-(1-xi[2])/4.0    -(1-xi[1])/4.0
                   (1-xi[2])/4.0    -(1+xi[1])/4.0
                   (1+xi[2])/4.0     (1+xi[1])/4.0
                  -(1+xi[2])/4.0     (1-xi[1])/4.0]

    ipoints = 1/sqrt(3)*[-1 -1; 1 -1; 1 1; -1 1]
    iweights = [1, 1, 1, 1]
    free_dofs = [3, 4, 5, 6]

    for i=1:10
        calc_local_matrices!(X, u, R, K, basis, dbasis, la, mu, ipoints, iweights)
        du[free_dofs] = K[free_dofs, free_dofs] \ -(R - F)[free_dofs]
        u += du
        if norm(du) < 1.0e-9
            Logging.debug("Converged in $i iterations.")
            break
        end
    end

    # Tested against Elmer solution
    Logging.debug("solution vector: \n $u")
    @fact u[2, 3] => roughly(-2.222244754401764)
    norm1 = norm(u)
    Logging.debug("norm of u: $(norm(u))")

    # We rotate model a bit and make sure that L2 norm is same
    phi = 30/180*pi
    rmat = [
        cos(phi) -sin(phi)
        sin(phi)  cos(phi)]
    X = rmat*X
    F = rmat*F
    u = zeros(2, 4)
    for i=1:10
        calc_local_matrices!(X, u, R, K, basis, dbasis, la, mu, ipoints, iweights)
        du[free_dofs] = K[free_dofs, free_dofs] \ -(R - F)[free_dofs]
        u += du
        if norm(du) < 1.0e-9
            Logging.debug("Converged in $i iterations.")
            break
        end
    end
    Logging.debug("solution vector: \n $u")
    Logging.debug("norm of u: $(norm(u))")
    @fact norm(u) => roughly(norm1) 
end

test solve one element model


10-Aug 19:18:44:DEBUG:root:Converged in 6 iterations.
10-Aug 19:18:45:DEBUG:root:solution vector: 
 [0.0 -0.39914506095474317 -0.07228582695592449 0.0
 0.0 -2.1779892317073504 -2.222244754401764 0.0]
10-Aug 19:18:45:DEBUG:root:norm of u: 3.1292483947150043
10-Aug 19:18:45:DEBUG:root:Converged in 6 iterations.
10-Aug 19:18:45:DEBUG:root:solution vector: 
 [0.0 0.7433248532717793 1.0485210147234858 0.0
 0.0 -2.085766534304891 -1.9606633242166027 0.0]
10-Aug 19:18:45:DEBUG:root:norm of u: 3.129248394715006


2 facts verified.


delayed_handler (generic function with 4 methods)

One element solutions are not particularly interesting so next step is to create function that assembles global matrix from local matrices. Some data types:

In [55]:
#type Node
#    id :: Int
#    #elements :: Array{Int64, 1}
#end

In [6]:
type Element
    id :: Int
    node_ids :: Array{Int64, 1}
    coordinates :: Array{Float64, 2}
    attributes :: Dict{ASCIIString, Any}
end

In [7]:
type Assembly
    # LHS
    I :: Array{Int64, 1}
    J :: Array{Int64, 1}
    A :: Array{Float64, 1}
    # RHS
    i :: Array{Int64, 1}
    b :: Array{Float64, 1}
    # global dofs for each element
    gdofs :: Dict{Int64, Array{Int64, 1}}
end

In [56]:
"""
Return shape functions and their derivatives for a element.

Parameters
----------
element::Element

Returns
-------
tuple (basis, dbasis)
"""
function get_shape_functions(el::Element)
    ndim, nnodes = size(el.coordinates)
    #Logging.debug("ndim = $ndim, nnodes=$nnodes")
    if (nnodes == 4) & (ndim == 2)
        basis(xi) = [
            (1-xi[1])*(1-xi[2])/4
            (1+xi[1])*(1-xi[2])/4
            (1+xi[1])*(1+xi[2])/4
            (1-xi[1])*(1+xi[2])/4]
        dbasis(xi) = [-(1-xi[2])/4.0    -(1-xi[1])/4.0
                       (1-xi[2])/4.0    -(1+xi[1])/4.0
                       (1+xi[2])/4.0     (1+xi[1])/4.0
                      -(1+xi[2])/4.0     (1-xi[1])/4.0]
        return basis, dbasis
    elseif (nnodes == 10) & (ndim == 3)
        basis(xi) = [(xi[1] + xi[2] + xi[3] - 1)*(2*xi[1] + 2*xi[2] + 2*xi[3] - 1)
                                        -xi[1]*(-2*xi[1] + 1)
                                        -xi[2]*(-2*xi[2] + 1)
                                        -xi[3]*(-2*xi[3] + 1)
                         4*xi[1]*(-xi[1] - xi[2] - xi[3] + 1)
                                                4*xi[1]*xi[2]
                         4*xi[2]*(-xi[1] - xi[2] - xi[3] + 1)
                                                4*xi[1]*xi[3]
                                                4*xi[2]*xi[3]
                         4*xi[3]*(-xi[1] - xi[2] - xi[3] + 1)]

        dbasis(xi) = [
            4*xi[1] + 4*xi[2] + 4*xi[3] - 3   4*xi[1] + 4*xi[2] + 4*xi[3] - 3   4*xi[1] + 4*xi[2] + 4*xi[3] - 3
                             4*xi[1] - 1                                0                                0
                                       0                      4*xi[2] - 1                                0
                                       0                                0                      4*xi[3] - 1
        -8*xi[1] - 4*xi[2] - 4*xi[3] + 4                         -4*xi[1]                         -4*xi[1]
                                 4*xi[2]                          4*xi[1]                                0
                                -4*xi[2]  -4*xi[1] - 8*xi[2] - 4*xi[3] + 4                         -4*xi[2]
                                 4*xi[3]                                0                          4*xi[1]
                                       0                          4*xi[3]                          4*xi[2]
                                -4*xi[3]                         -4*xi[3] -4*xi[1] - 4*xi[2] - 8*xi[3] + 4]
        return basis, dbasis
    end
    throw("Unknown function space, ndim=$ndim, nnodes=$nnodes")
end

"""
"""
function get_integration_scheme(el::Element, order=2)
    ndim, nnodes = size(el.coordinates)
    if (nnodes == 4) & (order == 2) & (ndim == 2)
        ipoints = 1/sqrt(3)*[-1 -1; 1 -1; 1 1; -1 1]
        iweights = [1, 1, 1, 1]
        return ipoints, iweights
    elseif (nnodes == 10) & (order == 2) & (ndim == 3) # c3d10
        # from code aster documentation
        a = 1/20*(5-sqrt(5))
        b = 1/20*(5+3*sqrt(5))
        ipoints = [a a a; a a b; a b a; b a a]
        iweights = 1/24*[1 1 1 1]
        return ipoints, iweights
    end
end

get_integration_scheme (generic function with 2 methods)

In [79]:
function assemble_element!(ass::Assembly, el::Element, io=2)

    # Material properties
    E = el.attributes["Young"]
    nu = el.attributes["Poisson"]
    mu = E/(2*(1+nu))
    la = E*nu/((1+nu)*(1-2*nu))
    la = 2*la*mu/(la + 2*mu)

    dofs = prod(size(el.coordinates))
    X = el.coordinates
    u = el.attributes["displacement"]
    R = el.attributes["displacement nodal force"]
    K = el.attributes["displacement tangent stiffness"]

    gdofs = ass.gdofs[el.id]
    Logging.debug("Assemble element to gdofs $gdofs")
    basis, dbasis = get_shape_functions(el)
    ipoints, iweights = get_integration_scheme(el, io)
    calc_local_matrices!(X, u, R, K, basis, dbasis, la, mu, ipoints, iweights)

    for i=1:dofs
        for j=1:dofs
            push!(ass.I, gdofs[i])
            push!(ass.J, gdofs[j])
            push!(ass.A, K[i,j])
        end
        push!(ass.i, gdofs[i])
        push!(ass.b, R[i])
    end
end

assemble_element! (generic function with 2 methods)

Time to test again. From last test we know that correct solution is

    [0.0 -0.39914506095474317 -0.07228582695592449 0.0
     0.0 -2.1779892317073504  -2.222244754401764   0.0]

This time we assemble global stiffness matrix in different order, 2 3 4 1

In [58]:
facts("one element assembly") do
    # Create model
    #Logging.debug("Creating nodes")
    #n1 = Node(1)
    #n2 = Node(2)
    #n3 = Node(3)
    #n4 = Node(4)
    Logging.debug("Adding nodes to array")
    #nodes = [n1.id, n2.id, n3.id, n4.id]
    node_ids = [1, 2, 3, 4]
    coordinates = [10.0 0.0; 10.0 1.0; 0.0 1.0; 0.0 0.0]'
    attributes = Dict("Young" => 90, "Poisson" => 0.25)
    Logging.debug("Creating elements")
    el = Element(1, node_ids, coordinates, attributes)

    # Initialize elements ready for solution
    el.attributes["displacement"] = zeros(2, 4)
    el.attributes["displacement nodal force"] = zeros(2, 4)
    el.attributes["displacement tangent stiffness"] = zeros(8, 8)

    for i=1:10
        Logging.debug("Starting iteration $i")
        ass = Assembly(Int64[], Int64[], Float64[], Int64[], Float64[], Dict{Int64,Array{Int64,1}}())
        ass.gdofs[el.id] = [1, 2, 3, 4, 5, 6, 7, 8]
        Logging.debug("Assembling")
        assemble_element!(ass, el)

        # Boundary conditions
        F = [0 0; 0 -2; 0 0; 0 0]'
        F = reshape(F, prod(size(F)))
        free_dofs = [1, 2, 3, 4]

        # solution
        K = sparse(ass.I, ass.J, ass.A)
        R = full(sparsevec(ass.i, ass.b))
        R = R - F
        du = zeros(8) # must be determined from ass
        du[free_dofs] = K[free_dofs, free_dofs] \ -R[free_dofs]

        Logging.debug("Solution norm = $(norm(du))")

        # update solution back to elements
        eldu = du[ass.gdofs[el.id]]
        eldu = reshape(eldu, (2, round(Int, length(eldu)/2)))
        el.attributes["displacement"] += eldu
        if norm(du) < 1.0e-9
            Logging.debug("Converged in $i iterations.")
            break
        end
    end
    disp = el.attributes["displacement"]
    Logging.debug("Displacement of element = \n$disp")
    @fact norm(disp) => roughly(3.1292483947150043)
end

one element assembly


10-Aug 21:03:06:DEBUG:root:Adding nodes to array
10-Aug 21:03:06:DEBUG:root:Creating elements
10-Aug 21:03:06:DEBUG:root:Starting iteration 1
10-Aug 21:03:06:DEBUG:root:Assembling
10-Aug 21:03:06:DEBUG:root:Solution norm = 3.090022136728999
10-Aug 21:03:06:DEBUG:root:Starting iteration 2
10-Aug 21:03:06:DEBUG:root:Assembling
10-Aug 21:03:06:DEBUG:root:Solution norm = 0.3212131602153504
10-Aug 21:03:06:DEBUG:root:Starting iteration 3
10-Aug 21:03:06:DEBUG:root:Assembling
10-Aug 21:03:06:DEBUG:root:Solution norm = 0.040431781940005365
10-Aug 21:03:06:DEBUG:root:Starting iteration 4
10-Aug 21:03:06:DEBUG:root:Assembling
10-Aug 21:03:06:DEBUG:root:Solution norm = 0.0009291101052064257
10-Aug 21:03:06:DEBUG:root:Starting iteration 5
10-Aug 21:03:06:DEBUG:root:Assembling
10-Aug 21:03:06:DEBUG:root:Solution norm = 1.5638899025378415e-7
10-Aug 21:03:06:DEBUG:root:Starting iteration 6
10-Aug 21:03:06:DEBUG:root:Assembling
10-Aug 21:03:06:DEBUG:root:Solution norm = 1.0355045356935844e-14
10-Aug 

delayed_handler (generic function with 4 methods)

1 fact verified.


Seems to be working. But we still need to handle boundary conditions more "cleverly" and generalize assembly to several elements (which is not problem).

First of all, essential boundary conditions are nothing more than equality constraints saying that value for some degree of freedom is fixed. Elimination is just a special case when this value equals to zero. There is couple of different strategies to handle essential boundary conditions. One option is to force them using Lagrange multipliers which can also be used to create all kind of kinematic constraints also. (For example, contact can be considered as a kinematic constraint.) Another option is to manipulate matrix such a way that constraint is satisfied.

Because we are now going "bottom-up", we develop something extremely simple that however deals with the problem:

In [14]:
type BC
    dofs :: Array{Int64, 1}
    values :: Array{Float64, 1}
end

In [82]:
"""
Create local dof to global dof mapping for given elements
"""
function create_ldof2gdofmap(elements; ndofs=2)
    Logging.info("create_ldof2gdofmap: dofs per node: $ndofs")

    all_node_ids = Int64[]
    for el in elements
        eldim, elnodes = size(el.coordinates)
        for nid in el.node_ids
            push!(all_node_ids, nid)
        end
    end
    all_node_ids = unique(all_node_ids)
    sort!(all_node_ids)

    # Assign global dof for each node
    pdim = 1
    ngdofs = Dict{Int64, Array{Int64,1}}()
    for nid in all_node_ids
        ngdofs[nid] = collect(pdim:pdim+ndofs-1)
        pdim += ndofs
    end

    return ngdofs
end

function solve!(elements, dofmap, neumann_bcs, dirichlet_bcs; ndofs=2, max_iterations=10)
    
    Logging.info("solve!: dofs per node: $ndofs")
    pdim = length(dofmap)*ndofs
    Logging.debug("Problem size = $pdim")

    # Initialize elements ready for solution
    for el in elements
        eldim, elnodes = size(el.coordinates)
        el.attributes["displacement"] = zeros(ndofs, elnodes)
        el.attributes["displacement nodal force"] = zeros(ndofs, elnodes)
        el.attributes["displacement tangent stiffness"] = zeros(ndofs*elnodes, ndofs*elnodes)
    end

    # Assign global dofs for elements
    gdofs = Dict{Int64, Array{Int64,1}}()
    for el in elements
        gdofs[el.id] = Int64[]
        for nid in el.node_ids
            for ndof in dofmap[nid]
                push!(gdofs[el.id], ndof)
            end
        end
    end

    ass = Assembly(Int64[], Int64[], Float64[], Int64[], Float64[], gdofs)

    for iter=1:max_iterations
        Logging.debug("Starting iteration $iter")
        ass.I = []
        ass.J = []
        ass.A = []
        ass.i = []
        ass.b = []
        
        Logging.debug("Assembling")
        for el in elements
            assemble_element!(ass, el)
            println("Element stiffness matrix")
            dump(round(el.attributes["displacement tangent stiffness"], 2))
        end

        Logging.debug("Adding Dirichlet boundary conditions")
        # Dirichlet boundary conditions
        i = 1
        for bc in dirichlet_bcs
            for (dof, val) in zip(bc.dofs, bc.values)
                Logging.debug("dof $dof => $val")
                push!(ass.I, dof)
                push!(ass.J, pdim+i)
                push!(ass.A, 1)
                push!(ass.I, pdim+i)
                push!(ass.J, dof)
                push!(ass.A, 1)
                push!(ass.i, pdim+i)
                push!(ass.b, 0)
                i += 1
            end
        end
        i -= 1
        Logging.debug("Added $i Lagrange multipliers to model")

        Logging.debug("Adding Neumann boundary conditions")
        F = zeros(pdim+i)
        # Neumann boundary conditions
        for bc in neumann_bcs
            for (dof, val) in zip(bc.dofs, bc.values)
                F[dof] += val
            end
        end

        Logging.debug("Solving system of equations. Total size = $(pdim+i)")
        # solution
        K = sparse(ass.I, ass.J, ass.A)
        R = full(sparsevec(ass.i, ass.b))
        R = R - F
        Logging.debug(dump(round(full(K), 1)))
        #print_matrix(full(K))
        Logging.debug(dump(round(R', 1)))

        du = K \ -R

        solnorm = norm(du[1:pdim])
        Logging.debug("Solution norm = $solnorm")

        # update solution back to elements
        for el in elements

            eldu = du[ass.gdofs[el.id]]
            eldu = reshape(eldu, (ndofs, round(Int, length(eldu)/ndofs)))
            el.attributes["displacement"] += eldu
        end
        if solnorm < 1.0e-9
            Logging.debug("Converged in $iter iterations.")
            break
        end
    end

end

ENV["COLUMNS"] = 160

facts("solve one element problem") do
    # Create model
    Logging.debug("Creating nodes")
    node_ids = [1, 2, 3, 4]
    coordinates = [10.0 0.0; 10.0 1.0; 0.0 1.0; 0.0 0.0]'
    attributes = Dict("Young" => 90, "Poisson" => 0.25)
    Logging.debug("Creating elements")
    el = Element(1, node_ids, coordinates, attributes)
    elements = [el]
    dofmap = create_ldof2gdofmap(elements)
    Logging.debug(dofmap)
    # Boundary conditions
    # here we want to create nodal force for third dof, that is, node id 2, second dof
    bc1 = BC([dofmap[2][2]], [-2.0])
    # dirichlet bc, set dx=dy=0 on support
    bc2 = BC([dofmap[3][1], dofmap[3][2], dofmap[4][1], dofmap[4][2]], [0.0, 0.0, 0.0, 0.0])
    solve!(elements, dofmap, [bc1], [bc2]; max_iterations=7)
    disp = elements[1].attributes["displacement"]
    Logging.debug("Displacement of element = \n$disp")
    @fact norm(disp) => roughly(3.1292483947150043)
end

facts("solve two element problem") do
    # Create model
    attributes = Dict("Young" => 90, "Poisson" => 0.25)

    Logging.debug("Creating elements")
    nids1 = [1, 2, 5, 4]
    coords1 = [0.0 0.0; 5.0 0.0; 5.0 1.0; 0.0 1.0]'
    el1 = Element(1, nids1, coords1, attributes)
    nids2 = [2, 3, 6, 5]
    coords2 = [5.0 0.0; 10.0 0.0; 10.0 1.0; 5.0 1.0]'
    el2 = Element(2, nids2, coords2, attributes)
    elements = [el1, el2]

    dofmap = create_ldof2gdofmap(elements)
    Logging.debug(dofmap)
    # Boundary conditions
    # here we want to create nodal force for third dof, that is, node id 2, second dof
    bc1 = BC([dofmap[6][2]], [-2.0])
    # dirichlet bc, set dx=dy=0 on support
    bc2 = BC([dofmap[1][1], dofmap[1][2], dofmap[4][1], dofmap[4][2]], [0.0, 0.0, 0.0, 0.0])
    solve!(elements, dofmap, [bc1], [bc2]; max_iterations=7)
    disp = elements[2].attributes["displacement"]
    Logging.debug("Displacement of element = \n$disp")
    #@fact norm(disp) => roughly(3.1292483947150043)
end

solve one element problem


10-Aug 22:55:49:DEBUG:root:Creating nodes
10-Aug 22:55:49:DEBUG:root:Creating elements
10-Aug 22:55:49:INFO:root:create_ldof2gdofmap: dofs per node: 2
10-Aug 22:55:49:DEBUG:root:Dict(4=>[7,8],2=>[3,4],3=>[5,6],1=>[1,2])
10-Aug 22:55:50:INFO:root:solve!: dofs per node: 2
10-Aug 22:55:50:DEBUG:root:Problem size = 8
10-Aug 22:55:50:DEBUG:root:Starting iteration 1
10-Aug 22:55:50:DEBUG:root:Assembling
10-Aug 22:55:50:DEBUG:root:Assemble element to gdofs [1,2,3,4,5,6,7,8]


Element stiffness matrix


10-Aug 22:55:50:DEBUG:root:Adding Dirichlet boundary conditions
10-Aug 22:55:50:DEBUG:root:dof 5 => 0.0
10-Aug 22:55:50:DEBUG:root:dof 6 => 0.0
10-Aug 22:55:50:DEBUG:root:dof 7 => 0.0
10-Aug 22:55:50:DEBUG:root:dof 8 => 0.0
10-Aug 22:55:50:DEBUG:root:Added 4 Lagrange multipliers to model
10-Aug 22:55:50:DEBUG:root:Adding Neumann boundary conditions
10-Aug 22:55:50:DEBUG:root:Solving system of equations. Total size = 12
10-Aug 22:55:50:DEBUG:root:nothing
10-Aug 22:55:50:DEBUG:root:nothing
10-Aug 22:55:50:DEBUG:root:Solution norm = 3.0900221367289444
10-Aug 22:55:50:DEBUG:root:Starting iteration 2
10-Aug 22:55:50:DEBUG:root:Assembling
10-Aug 22:55:50:DEBUG:root:Assemble element to gdofs [1,2,3,4,5,6,7,8]
10-Aug 22:55:50:DEBUG:root:Adding Dirichlet boundary conditions
10-Aug 22:55:50:DEBUG:root:dof 5 => 0.0
10-Aug 22:55:50:DEBUG:root:dof 6 => 0.0
10-Aug 22:55:50:DEBUG:root:dof 7 => 0.0
10-Aug 22:55:50:DEBUG:root:dof 8 => 0.0
10-Aug 22:55:50:DEBUG:root:Added 4 Lagrange multipliers to model

Array(Float64,(8,8)) 8x8 Array{Float64,2}:
  123.2   -15.0  -118.4   -03.0   -61.6    15.0    56.8    03.0
  -15.0   321.2    03.0  -319.4    15.0  -160.6   -03.0   158.8
 -118.4    03.0   123.2    15.0    56.8   -03.0   -61.6   -15.0
  -03.0  -319.4    15.0   321.2    03.0   158.8   -15.0  -160.6
  -61.6    15.0    56.8    03.0   123.2   -15.0  -118.4   -03.0
   15.0  -160.6   -03.0   158.8   -15.0   321.2    03.0  -319.4
   56.8   -03.0   -61.6   -15.0  -118.4    03.0   123.2    15.0
   03.0   158.8   -15.0  -160.6   -03.0  -319.4    15.0   321.2
Array(Float64,(12,12)) 12x12 Array{Float64,2}:
  123.2   -15.0  -118.4   -03.0   -61.6    15.0    56.8    03.0  0.0  0.0  0.0  0.0
  -15.0   321.2    03.0  -319.4    15.0  -160.6   -03.0   158.8  0.0  0.0  0.0  0.0
 -118.4    03.0   123.2    15.0    56.8   -03.0   -61.6   -15.0  0.0  0.0  0.0  0.0
  -03.0  -319.4    15.0   321.2    03.0   158.8   -15.0  -160.6  0.0  0.0  0.0  0.0
  -61.6    15.0    56.8    03.0   123.2   -15.0  -118.4   -03.

...

10-Aug 22:55:50:DEBUG:root:nothing
10-Aug 22:55:50:DEBUG:root:nothing
10-Aug 22:55:50:DEBUG:root:Solution norm = 2.638889456564507
10-Aug 22:55:50:DEBUG:root:Starting iteration 6
10-Aug 22:55:50:DEBUG:root:Assembling
10-Aug 22:55:50:DEBUG:root:Assemble element to gdofs [1,2,3,4,9,10,7,8]
10-Aug 22:55:50:DEBUG:root:Assemble element to gdofs [3,4,5,6,11,12,9,10]
10-Aug 22:55:50:DEBUG:root:Adding Dirichlet boundary conditions
10-Aug 22:55:50:DEBUG:root:dof 1 => 0.0
10-Aug 22:55:50:DEBUG:root:dof 2 => 0.0
10-Aug 22:55:50:DEBUG:root:dof 7 => 0.0
10-Aug 22:55:50:DEBUG:root:dof 8 => 0.0
10-Aug 22:55:50:DEBUG:root:Added 4 Lagrange multipliers to model
10-Aug 22:55:50:DEBUG:root:Adding Neumann boundary conditions
10-Aug 22:55:50:DEBUG:root:Solving system of equations. Total size = 16
10-Aug 22:55:50:DEBUG:root:nothing
10-Aug 22:55:50:DEBUG:root:nothing
10-Aug 22:55:50:DEBUG:root:Solution norm = 2.5846689771724556
10-Aug 22:55:50:DEBUG:root:Starting iteration 7
10-Aug 22:55:50:DEBUG:root:Assembl

:
  80.6   18.4    54.0   14.3     0.0    0.0  -70.3  -25.4   -64.3   -7.3     0.0    0.0  1.0  0.0  0.0  0.0
  18.4   29.5    19.2   20.6     0.0    0.0  -30.0  -33.3    -7.6  -16.8     0.0    0.0  0.0  1.0  0.0  0.0
  54.0   19.2   220.1   65.6    54.0   14.3  -46.8  -24.3  -217.0  -67.5   -64.3   -7.3  0.0  0.0  0.0  0.0
  14.3   20.6    65.6  104.0    19.2   20.6  -24.7  -35.5   -66.8  -92.8    -7.6  -16.8  0.0  0.0  0.0  0.0
   0.0    0.0    54.0   19.2   139.5   47.2    0.0    0.0   -46.8  -24.3  -146.7  -42.1  0.0  0.0  0.0  0.0
   0.0    0.0    14.3   20.6    47.2   74.5    0.0    0.0   -24.7  -35.5   -36.8  -59.6  0.0  0.0  0.0  0.0
 -70.3  -30.0   -46.8  -24.7     0.0    0.0   65.4   33.7    51.7   21.0     0.0    0.0  0.0  0.0  1.0  0.0
 -25.4  -33.3   -24.3  -35.5     0.0    0.0   33.7   49.0    16.0   19.9     0.0    0.0  0.0  0.0  0.0  1.0
 -64.3   -7.6  -217.0  -66.8   -46.8  -24.7   51.7   16.0   224.7   62.2    51.7   21.0  0.0  0.0  0.0  0.0
  -7.3  -16.8   -67.5  -92

10-Aug 22:55:51:DEBUG:root:nothing
10-Aug 22:55:51:DEBUG:root:nothing
10-Aug 22:55:51:DEBUG:root:Solution norm = 2.4116101594574446
10-Aug 22:55:51:DEBUG:root:Displacement of element = 
[-3.0265017801117513 -5.464687844270099 -4.5022205763589715 -2.2188310526354926
 -1.2759688848713764 -6.757003969576785 -7.2218353000897295 -1.8649226742660745]


delayed_handler (generic function with 4 methods)

0.0   -69.3   -7.1  -141.9  -36.1    0.0    0.0    56.9   15.7   154.3   27.5  0.0  0.0  0.0  0.0
   0.0    0.0    -6.9  -18.7   -41.4  -59.1    0.0    0.0    20.9   21.6    27.5   56.1  0.0  0.0  0.0  0.0
   1.0    0.0     0.0    0.0     0.0    0.0    0.0    0.0     0.0    0.0     0.0    0.0  0.0  0.0  0.0  0.0
   0.0    1.0     0.0    0.0     0.0    0.0    0.0    0.0     0.0    0.0     0.0    0.0  0.0  0.0  0.0  0.0
   0.0    0.0     0.0    0.0     0.0    0.0    1.0    0.0     0.0    0.0     0.0    0.0  0.0  0.0  0.0  0.0
   0.0    0.0     0.0    0.0     0.0    0.0    0.0    1.0     0.0    0.0     0.0    0.0  0.0  0.0  0.0  0.0
Array(Float64,(1,16)) 1x16 Array{Float64,2}:
 22.9  4.1  28.6  11.8  5.6  7.6  -19.5  -12.4  -28.6  -11.8  -09.0  2.6  0.0  0.0  0.0  0.0
0 facts verified.


## Top-down design

Next we see this problem from "other direction", by parsing ABAQUS .inp file and making 3d simulation.

In [74]:
sparse([1, 1], [1, 1], [1.0, 1.0])

1x1 sparse matrix with 1 Float64 entries:
	[1, 1]  =  2.0

In [46]:
using JuliaFEM.abaqus_reader
fid = open("../geometry/3d_beam/palkki.inp")
model = JuliaFEM.abaqus_reader.parse_abaqus(fid)
close(fid)
model

10-Aug 20:59:55:INFO:root:Registered handlers: Any["ELEMENT","NODE","NSET"]
10-Aug 20:59:55:DEBUG:root:Found NODE section
10-Aug 20:59:56:DEBUG:root:Found ELEMENT section
10-Aug 20:59:57:DEBUG:root:120 elements found
10-Aug 20:59:57:INFO:root:Creating ELSET Body1
10-Aug 20:59:57:DEBUG:root:Found NSET section
10-Aug 20:59:57:DEBUG:root:Creating node set SUPPORT
10-Aug 20:59:57:DEBUG:root:Found NSET section
10-Aug 20:59:57:DEBUG:root:Creating node set LOAD
10-Aug 20:59:57:DEBUG:root:Found NSET section
10-Aug 20:59:57:DEBUG:root:Creating node set TOP


Dict{Any,Any} with 4 entries:
  "nodes"    => Dict{Any,Any}(288=>[97.5,7.5,10.0],11=>[92.5,2.5,5.0],134=>[45.…
  "elements" => Dict{Any,Any}(68=>[71,144,149,198,51,150,57,43,50,214],2=>[204,…
  "elsets"   => Dict{Any,Any}("Body1"=>[1,2,3,4,5,6,7,8,9,10  …  111,112,113,11…
  "nsets"    => Dict{Any,Any}("LOAD"=>[82,84,87,179,197,246,249,256,257],"SUPPO…

In [63]:
facts("solve 3d elasticity problem") do

    attributes = Dict("Young" => 90, "Poisson" => 0.25)

    Logging.debug("Creating elements")
    elements = Element[]
    coordinates = zeros(3, 10)
    for (elid, node_ids) in model["elements"]
        for (i, nid) in enumerate(node_ids)
            coordinates[:,i] = model["nodes"][nid]
        end
        el = Element(elid, node_ids, coordinates, attributes)
        push!(elements, el)
    end

    # create "dofmap" so that we know how to assemble global stiffness matrix
    dofmap = create_ldof2gdofmap(elements; ndofs=3)

    # Boundary conditions

    # dirichlet bc, set dx=dy=dz for all nodes in set SUPPORT
    bc_support = BC(Int64[], Float64[])
    for nid in model["nsets"]["SUPPORT"]
        for i=1:3
            push!(bc_support.dofs, dofmap[nid][i])
            push!(bc_support.values, 0.0)
        end
    end

    # force boundary condition, put -1 to 2nd dof for each node in set LOAD
    bc_load = BC(Int64[], Float64[])
    for nid in model["nsets"]["LOAD"]
        push!(bc_load.dofs, dofmap[nid][2])
        push!(bc_load.values, -0.1)
    end

    #solve!(elements, [bc1], [bc2]; max_iterations=7)
    neumann_bcs = [bc_load]
    dirichlet_bcs = [bc_support]
    # ndofs = dimension of unknown field in nodes
    solve!(elements, dofmap, neumann_bcs, dirichlet_bcs; ndofs=3, max_iterations=5)
#    disp = elements[1].attributes["displacement"]
#    Logging.debug("Displacement of element = \n$disp")
#    @fact norm(disp) => roughly(3.1292483947150043)
end

solve 3d elasticity problem


10-Aug 21:16:13:DEBUG:root:Creating elements
10-Aug 21:16:13:INFO:root:dofs per node: 3
10-Aug 21:16:13:INFO:root:dofs per node: 3
10-Aug 21:16:13:DEBUG:root:Problem size = 894
10-Aug 21:16:13:DEBUG:root:Starting iteration 1
10-Aug 21:16:13:DEBUG:root:Assembling
10-Aug 21:16:15:DEBUG:root:Adding Dirichlet boundary conditions
10-Aug 21:16:15:DEBUG:root:Added 27 Lagrange multipliers to model
10-Aug 21:16:15:DEBUG:root:Adding Neumann boundary conditions
10-Aug 21:16:15:DEBUG:root:Solving system of equations. Total size = 921
10-Aug 21:16:15:DEBUG:root:Solution norm = 0.2429242363684311
10-Aug 21:16:15:DEBUG:root:Starting iteration 2
10-Aug 21:16:15:DEBUG:root:Assembling
10-Aug 21:16:16:DEBUG:root:Adding Dirichlet boundary conditions
10-Aug 21:16:16:DEBUG:root:Added 27 Lagrange multipliers to model
10-Aug 21:16:16:DEBUG:root:Adding Neumann boundary conditions
10-Aug 21:16:16:DEBUG:root:Solving system of equations. Total size = 921
10-Aug 21:16:16:DEBUG:root:Solution norm = 0.22033994164829

0 facts verified.


delayed_handler (generic function with 4 methods)

In [4]:
xdoc, model = JuliaFEM.xdmf.xdmf_new_model()
temporal_collection = JuliaFEM.xdmf.xdmf_new_temporal_collection(model)
grid = JuliaFEM.xdmf.xdmf_new_grid(temporal_collection; time=0)
JuliaFEM.xdmf.xdmf_new_mesh(grid, X3d, elmap2)
JuliaFEM.xdmf.xdmf_new_field(grid, "Displacement", "nodes", u3d)
print(xdoc)
JuliaFEM.xdmf.xdmf_save_model(xdoc, "/tmp/foo.xmf")

<?xml version="1.0" encoding="utf-8"?>
<Xdmf xmlns:xi="http://www.w3.org/2001/XInclude" Version="2.1">
  <Domain>
    <Grid CollectionType="Temporal" GridType="Collection" Name="Collection">
      <Geometry Type="None"/>
      <Topology Dimensions="0" Type="NoTopology"/>
      <Grid Name="Grid">
        <Time Value="0"/>
        <Geometry Type="XYZ">
          <DataItem DataType="Float" Dimensions="12" Format="XML" Precision="4">0.0 0.0 0.0 10.0 0.0 0.0 10.0 1.0 0.0 0.0 1.0 0.0</DataItem>
        </Geometry>
        <Topology Dimensions="1" Type="Mixed">
          <DataItem DataType="Int" Dimensions="5" Format="XML" Precision="4">5 0 1 2 3</DataItem>
        </Topology>
        <Attribute Center="Node" Name="Displacement" Type="Vector">
          <DataItem DataType="Float" Dimensions="12" Format="XML" Precision="4">0.0 0.0 0.0 -0.3991450609547433 -2.17798923170735 0.0 -0.07228582695592455 -2.222244754401764 0.0 0.0 0.0 0.0</DataItem>
        </Attribute>
      </Grid>
    </Grid>
  </D

1015

 in depwarn at /Applications/Julia-0.4.0-dev-539c818c4e.app/Contents/Resources/julia/lib/julia/sys.dylib
 in int at deprecated.jl:49
 in save_file at /Users/jukka/.julia/v0.4/LightXML/src/document.jl:108
 in xdmf_save_model at /Users/jukka/.julia/v0.4/JuliaFEM/src/xdmf.jl:106
 in include_string at loading.jl:99
 in execute_request_0x535c5df2 at /Users/jukka/.julia/v0.4/IJulia/src/execute_request.jl:157
 in eventloop at /Users/jukka/.julia/v0.4/IJulia/src/IJulia.jl:123
 in anonymous at task.jl:365
while loading In[4], in expression starting on line 7


## 3d beam with quadratic elements

In [5]:
nnodes = length(model["nodes"])
nelements = length(model["elements"])
dim = 3
E = 90
nu = 0.25
mu = E/(2*(1+nu))
la = E*nu/((1+nu)*(1-2*nu))

X = zeros(dim, nnodes)
u = zeros(dim, nnodes)
du = zeros(dim, nnodes)
elmap = zeros(Int, 10, nelements)
nodalloads = zeros(3, nnodes)
dirichletbc = NaN*ones(3, nnodes)
la = la*ones(1, nnodes)
mu = mu*ones(1, nnodes)

# calculate permutation which maps node ids to matrix indices
perm = Dict()
for (j, k) in enumerate(keys(model["nodes"]))
    perm[k] = j
end

for j=1:nnodes
    #X[:,j] = model["nodes"][perm[j]]
    X[:,j] = model["nodes"][j]
end

for (j, k) in enumerate(keys(model["elements"]))
    #node_ids = model["elements"][j]
    elmap[:,j] = model["elements"][j]
    #for l=1:10
    #    elmap[l, j] = perm[node_ids[l]]
    #end
end

elmap

10x120 Array{Int64,2}:
 243  204  259  145   96   96  236  285  …  217   69  154  179  203   96  259
 240  199   70  175   88  101   88  179     216  144  114   91  204  267  199
 191  175   69  199  236  164  285  178     278   78  278  178  259   95  204
 117  130  130  130  178   97  178   83     155   71  218   83  199   97  130
 245  207  265  177  141  102  290   12     219  146   20  181  206  268   39
 242  208   72  208  290  171  289  182  …  282  152  280  180  263  272  207
 244  209    5  202  291    9  287   11      33   79   32  182  262   98  263
   1    3    6  174    7   99  237   13     224   74  223   14  205   99    6
   2    4  132  176    8  103    8   14     225   51  284   93  207   24    4
 196  176  134    4  237   10   11   15      17   80  283   15   39  100    3

In [6]:
# Handle dirichlet boundaries on SUPPORT
for j in model["nsets"]["SUPPORT"]
    #dirichletbc[perm[j]] = 0.0
    dirichletbc[j] = 0.0
end
dirichletbc

3x298 Array{Float64,2}:
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  …  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN

Shape functions and integration points

Add point force to LOAD nodeset

In [10]:
model["nsets"]["LOAD"]

9-element Array{Int64,1}:
  82
  84
  87
 179
 197
 246
 249
 256
 257

In [11]:
#nodalloads[3, perm[82]] = -0.06
nodalloads[3, 82] = -50.0
println(model["nodes"][82])

[100.0,10.0,0.0]


In [12]:
u = zeros(dim, nnodes)
du = zeros(dim, nnodes)

for i=1:10
    JuliaFEM.elasticity_solver.solve_elasticity_increment!(X, u, du, elmap, nodalloads, dirichletbc,
    la, mu, Ntet, dNtet, ipoints, iweights)
    u += du
    println("iteration $i, norm = $(norm(du))")
    if norm(du) < 1.0e-9
        println("Converged")
        break
    end
end
u

iteration 1, norm = 93.13879357262775
iteration 2, norm = 14.241640589907869
iteration 3, norm = 2.3999534862146534
iteration 4, norm = 0.8191634532858331
iteration 5, norm = 0.08574651168344295
iteration 6, norm = 0.00032306479346201524
iteration 7, norm = 8.052462405199464e-9
iteration 8, norm = 3.0878906643174355e-14


3x298 Array{Float64,2}:
 -0.0479657  -0.0483971  -0.0497284  …  -0.0155295  -0.0150513  -0.0462958
 -0.474054   -0.418817   -0.254176      -0.229641   -0.283098   -0.694379 
  0.232677    0.198051    0.0948355      0.128246    0.162511    0.371043 

Converged


In [13]:
maximum(abs(u))

15.449170689704438

In [14]:
size(elmap)

(10,120)

In [15]:
nelements

120

In [16]:
elcodes = 0x0026*ones(Int, nelements)
elmap2 = [elcodes'
          elmap]

11x120 Array{Int64,2}:
  38   38   38   38   38   38   38   38  …   38   38   38   38   38   38   38
 243  204  259  145   96   96  236  285     217   69  154  179  203   96  259
 240  199   70  175   88  101   88  179     216  144  114   91  204  267  199
 191  175   69  199  236  164  285  178     278   78  278  178  259   95  204
 117  130  130  130  178   97  178   83     155   71  218   83  199   97  130
 245  207  265  177  141  102  290   12  …  219  146   20  181  206  268   39
 242  208   72  208  290  171  289  182     282  152  280  180  263  272  207
 244  209    5  202  291    9  287   11      33   79   32  182  262   98  263
   1    3    6  174    7   99  237   13     224   74  223   14  205   99    6
   2    4  132  176    8  103    8   14     225   51  284   93  207   24    4
 196  176  134    4  237   10   11   15  …   17   80  283   15   39  100    3

In [17]:
xdoc, xmodel = JuliaFEM.xdmf.xdmf_new_model()
temporal_collection = JuliaFEM.xdmf.xdmf_new_temporal_collection(xmodel)
grid = JuliaFEM.xdmf.xdmf_new_grid(temporal_collection; time=0)
JuliaFEM.xdmf.xdmf_new_mesh(grid, X, elmap2)
#JuliaFEM.xdmf.xdmf_new_field(grid, "Displacement", "nodes", u)
#print(xdoc)
JuliaFEM.xdmf.xdmf_save_model(xdoc, "/tmp/foo3d2.xmf")

9568

 in depwarn at /Applications/Julia-0.4.0-dev-539c818c4e.app/Contents/Resources/julia/lib/julia/sys.dylib
 in int at deprecated.jl:49
 in save_file at /Users/jukka/.julia/v0.4/LightXML/src/document.jl:108
 in xdmf_save_model at /Users/jukka/.julia/v0.4/JuliaFEM/src/xdmf.jl:140
 in include_string at loading.jl:99
 in execute_request_0x535c5df2 at /Users/jukka/.julia/v0.4/IJulia/src/execute_request.jl:157
 in eventloop at /Users/jukka/.julia/v0.4/IJulia/src/IJulia.jl:123
 in anonymous at task.jl:365
while loading In[17], in expression starting on line 7


In [18]:
elmap[:,[1, 101]]

10x2 Array{Int64,2}:
 243  145
 240  199
 191   69
 117  130
 245  202
 242   47
 244  148
   1  174
   2    4
 196  134

In [25]:
tmp = elmap[:,[1]]
tmp = reshape(tmp, length(tmp))
X[:, tmp]

3x10 Array{Float64,2}:
 20.0  30.0  20.0  20.0  25.0  25.0  20.0  20.0  25.0  20.0
  0.0   0.0   0.0  10.0   0.0   0.0   0.0   5.0   5.0   5.0
 10.0  10.0   0.0   0.0  10.0   5.0   5.0   5.0   5.0   0.0

In [26]:
#tmpelmap = [38 1 2 3 4 5 6 7 8 9 10; 38 11 12 13 14 15 16 17 18 19 20]'
tmpelmap = [38 1 2 3 4 5 6 7 8 9 10]'

11x1 Array{Int64,2}:
 38
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [48]:
using LightXML

function xdmf_new_mesh(grid, X, elmap)
    dim, nnodes = size(X)
    geometry = new_child(grid, "Geometry")
    set_attribute(geometry, "Type", "XYZ")
    dataitem = new_child(geometry, "DataItem")
    set_attribute(dataitem, "DataType", "Float")
    set_attribute(dataitem, "Dimensions", "$nnodes $dim")
    set_attribute(dataitem, "Format", "XML")
    set_attribute(dataitem, "Precision", 8)
    #add_text(dataitem, join(X, " "))
    s = "\n"
    
    for i=1:nnodes
        s *= "\t\t" * join(X[:,i], " ") * "\n"
    end
    s *= "       "
    add_text(dataitem, s)

    elmap2 = copy(elmap)
    elmap2[2:end,:] -= 1
    dim, nelements = size(elmap2)

    topology = new_child(grid, "Topology")
    #set_attribute(topology, "Dimensions", "1")
    set_attribute(topology, "TopologyType", "Mixed")
    set_attribute(topology, "NumberOfElements", nelements)
    dataitem = new_child(topology, "DataItem")
    set_attribute(dataitem, "DataType", "Int")
    set_attribute(dataitem, "Dimensions", "$nelements $dim")
    set_attribute(dataitem, "Format", "XML")
    set_attribute(dataitem, "Precision", 8)
    s = "\n"
    for i=1:nelements
        s *= "\t\t" * join(elmap2[:,i], " ") * "\n"
    end
    add_text(dataitem, s)
    #add_text(dataitem, join(elmap2, " "))
    
end

xdoc, xmodel = JuliaFEM.xdmf.xdmf_new_model()
temporal_collection = JuliaFEM.xdmf.xdmf_new_temporal_collection(xmodel)
grid = JuliaFEM.xdmf.xdmf_new_grid(temporal_collection; time=0)
xdmf_new_mesh(grid, X, elmap2)
JuliaFEM.xdmf.xdmf_new_field(grid, "Displacement", "nodes", u)
print(xdoc)
JuliaFEM.xdmf.xdmf_save_model(xdoc, "/tmp/foo3d.xmf")

<?xml version="1.0" encoding="utf-8"?>
<Xdmf xmlns:xi="http://www.w3.org/2001/XInclude" Version="2.1">
  <Domain>
    <Grid CollectionType="Temporal" GridType="Collection" Name="Collection">
      <Geometry Type="None"/>
      <Topology Dimensions="0" Type="NoTopology"/>
      <Grid Name="Grid">
        <Time Value="0"/>
        <Geometry Type="XYZ">
          <DataItem DataType="Float" Dimensions="298 3" Format="XML" Precision="8">
		20.0 5.0 5.0
		25.0 5.0 5.0
		40.0 5.0 5.0
		42.5 5.0 2.5
		47.5 7.5 5.0
		42.5 7.5 5.0
		85.0 5.0 5.0
		90.0 5.0 5.0
		80.0 5.0 5.0
		77.5 5.0 2.5
		92.5 2.5 5.0
		97.5 2.5 5.0
		95.0 7.5 7.5
		97.5 5.0 2.5
		92.5 5.0 2.5
		97.5 7.5 5.0
		2.5 2.5 5.0
		5.0 2.5 2.5
		35.0 5.0 5.0
		10.0 5.0 5.0
		15.0 5.0 5.0
		55.0 2.5 7.5
		77.5 5.0 7.5
		75.0 7.5 7.5
		75.0 2.5 7.5
		75.0 5.0 5.0
		75.0 2.5 2.5
		75.0 7.5 2.5
		2.5 7.5 5.0
		7.5 7.5 5.0
		5.0 5.0 5.0
		7.5 2.5 5.0
		5.0 2.5 7.5
		30.0 5.0 5.0
		50.36999 4.749 5.47225
		50.18499 2.3745 7.73613
		47.6849

28476